# 하나은행에서 환율 정보 가져오기
- 1995년 01월 03부터 -> 어제까지(2025-04-08)
- 월-금 datetime clode에 물어봐서 해결
- selenium으로 날짜 입력 후 환율 조회
- pd.read_html
- mysql에 exchange_rate 데이터베이스(exchange_rate 테이블로 저장)

In [137]:
from datetime import datetime, timedelta

start_date = datetime(1995,1,3)
end_date = datetime(2025,4,8)
# print(start_date, end_date)

date_list = []
current_date = start_date

while current_date <= end_date:
    current_date += timedelta(days=1)
    if current_date.weekday() < 5:
        date_list.append(current_date.strftime('%Y-%m-%d'))
        
print(f"총 날짜 수: {len(date_list)}")
print(date_list[:10], end='\n')

date_list = date_list[:5]


총 날짜 수: 7896
['1995-01-04', '1995-01-05', '1995-01-06', '1995-01-09', '1995-01-10', '1995-01-11', '1995-01-12', '1995-01-13', '1995-01-16', '1995-01-17']


In [86]:
for i in date_list:
    print(i, type(i))

1995-01-03 <class 'str'>
1995-01-04 <class 'str'>
1995-01-05 <class 'str'>
1995-01-06 <class 'str'>
1995-01-07 <class 'str'>


In [62]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )

url = 'https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do'

driver.get(url)
wait = WebDriverWait(driver, 10) 

for date in date_list:
    date_input = wait.until(EC.presence_of_element_located((By.ID, "tmpInqStrDt")))
    date_input.clear()
    date_input.send_keys(date)
    element = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".btnBoxCenter a")))
    element.click()
    time.sleep(2)



# 하나은행에서 환율조회 부분 찾아 selenium으로 열기
- 날짜 입력부분 찾아 날짜 넣기
- 조회 버튼 찾아 버튼 클릭하기
- 조회된 데이터를 pd.read_html을 이용해 dataframe으로 생성
- 조회된 데이터 프레임에 조회 날짜 컬럼 만들어 날짜 추가
- 하나은행 환율 페이지 > F12 > 전문가모드에서 Network >  실행 > Fetch/XHR > DOC
- url 찾아서 확인(https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do)

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = Options()
options.add_experimental_option("detach", True)
options.add_argument('start_maximized')
options.add_argument('Chrome/134.0.0.0')
options.add_argument('lang=ko_KR')

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )


url = 'https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do'
driver.get(url)

In [3]:
# 날짜 입력
#tmpInqStrDt
date_input = driver.find_element(By.CSS_SELECTOR, '#tmpInqStrDt')
date_input.clear()
date_input.send_keys('20250407')
date_input.send_keys(Keys.ENTER)

# 조회버튼 클릭
#HANA_CONTENTS_DIV > div.btnBoxCenter > a
search_button = driver.find_element(By.CSS_SELECTOR, '#HANA_CONTENTS_DIV > div.btnBoxCenter > a')
search_button.click()


In [6]:
from io import StringIO
import pandas as pd

In [ ]:
#searchContentDiv > div.printdiv > table > colgroup

In [7]:
df = pd.read_html(StringIO(driver.find_element(By.CSS_SELECTOR, '.tblBasic.leftNone').get_attribute('outerHTML')))[0]
display(df)

통화       현찰                              송금           \
                 통화     사실 때            파실 때            보낼 때     받을 때   
                 통화       환율 Spread       환율 Spread     보낼 때     받을 때   
0            미국 USD  1496.23   1.75  1444.77   1.75  1484.90  1456.10   
1      일본 JPY (100)  1011.37   1.75   976.59   1.75  1003.72   984.24   
2            유로 EUR  1637.21   1.99  1573.33   1.99  1621.32  1589.22   
3            중국 CNY   210.19   5.00   190.19   5.00   202.19   198.19   
4            홍콩 HKD   193.01   1.97   185.57   1.97   191.18   187.40   
5            태국 THB    44.48   5.00    39.83   6.00    42.79    41.95   
6            대만 TWD    50.40  13.10    40.12  10.00     0.00     0.00   
7           필리핀 PHP    28.20  10.00    23.54   8.20    25.89    25.39   
8          싱가포르 SGD  1108.80   1.99  1065.54   1.99  1098.04  1076.30   
9            호주 AUD   898.33   1.97   863.63   1.97   889.78   872.18   
10    베트남 VND (100)     6.38  11.80     5.04  11.80     5.76     5.66   
11           영국 GBP  1909.50   1.97  1835.72   1.97  1891.33  1853.89   
12          캐나다 CAD  1052.89   1.97  1012.21   1.97  1042.87  1022.23   
13        말레이시아 MYR   352.52   7.40   303.96   7.40     0.00   324.96   
14          러시아 RUB    18.69   9.50    14.09  17.50    17.66    16.48   
15        남아공화국 ZAR    80.97   8.00    68.99   8.00    75.87    74.09   
16         노르웨이 NOK   138.50   3.30   126.44   5.70   135.42   132.74   
17         뉴질랜드 NZD   830.40   1.97   798.32   1.97   822.50   806.22   
18          덴마크 DKK   222.08   3.30   202.74   5.70   217.13   212.85   
19          멕시코 MXN    78.53  10.50    63.61  10.50    71.78    70.36   
20           몽골 MNT     0.00   0.00     0.00   0.00     0.00     0.00   
21          바레인 BHD  4150.71   6.40  3588.97   8.00  3940.06  3862.04   
22        방글라데시 BDT     0.00   0.00     0.00   0.00     0.00     0.00   
23          브라질 BRL   273.94  10.20   223.74  10.00     0.00   245.61   
24         브루나이 BND  1130.65   4.00  1021.94   6.00     0.00     0.00   
25      사우디아라비아 SAR   416.30   6.30   364.61   6.90   395.54   387.72   
26         스리랑카 LKR     0.00   0.00     0.00   0.00     0.00     0.00   
27          스웨덴 SEK   150.71   3.30   137.59   5.70   147.35   144.45   
28          스위스 CHF  1744.07   1.97  1676.69   1.97  1727.48  1693.28   
29    아랍에미리트공화국 AED   422.36   5.50   372.73   6.90   404.35   396.35   
30          알제리 DZD     0.00   0.00     0.00   0.00     0.00     0.00   
31           오만 OMR  4086.95   7.00  3552.21   7.00     0.00     0.00   
32          요르단 JOD  2258.64   8.90  1908.13   8.00     0.00     0.00   
33         이스라엘 ILS   428.09  10.00   358.05   8.00     0.00     0.00   
34          이집트 EGP     0.00   0.00     0.00   0.00     0.00     0.00   
35           인도 INR     0.00   0.00     0.00   0.00    17.31    16.93   
36  인도네시아 IDR (100)     9.79  10.00     8.01  10.00     8.98     8.82   
37           체코 CZK    69.04   8.50    57.92   9.00    64.34    62.94   
38           칠레 CLP     1.63  10.00     1.35  10.00     0.00     0.00   
39        카자흐스탄 KZT     0.00   0.00     0.00   0.00     0.00     0.00   
40          카타르 QAR     0.00   0.00     0.00   0.00     0.00     0.00   
41           케냐 KES     0.00   0.00     0.00   0.00     0.00     0.00   
42         콜롬비아 COP     0.00   0.00     0.00   0.00     0.00     0.00   
43         쿠웨이트 KWD  5083.36   6.50  4391.27   8.00  4820.84  4725.38   
44         탄자니아 TZS     0.00   0.00     0.00   0.00     0.00     0.00   
45         튀르키예 TRY     0.00   0.00     0.00   0.00    39.10    38.26   
46         파키스탄 PKR     0.00   0.00     0.00   0.00     0.00     0.00   
47          폴란드 PLN   403.52   8.00   343.74   8.00   377.73   369.53   
48          헝가리 HUF     4.29   9.30     3.62   8.00     3.97     3.89   
49           네팔 NPR     0.00   0.00     0.00   0.00     0.00     0.00   
50          마카오 MOP     0.00   0.00     0.00   0.00     0.00     0.00   
51         캄보디아 KHR     0.00   0.00     0.00   0.00     0.00     0.00   
52

In [11]:
date_input = driver.find_element(By.CSS_SELECTOR, '#tmpInqStrDt')
date_input.clear()
date_input.send_keys('20250401')
date_input.send_keys(Keys.ENTER)

search_button = driver.find_element(By.CSS_SELECTOR, '#HANA_CONTENTS_DIV > div.btnBoxCenter > a')
search_button.click()

# 날짜 생성
- 1995-01-03 ~ 2025-04-08 날짜 생성
- 평일만 날짜 생성
- 날짜 리스트에 날짜 데이터 저장


In [15]:
from datetime import datetime, timedelta

In [13]:
current_date = datetime(1995,1,3)

In [16]:
# weekday() : 월(0)~일(6)
for days in range(1,10):
#     print(current_date + datetime.timedelta(days=days))
    day = current_date + timedelta(days=days)
    if day.weekday() < 5:
        print(day, day.weekday())

1995-01-04 00:00:00 2
1995-01-05 00:00:00 3
1995-01-06 00:00:00 4
1995-01-09 00:00:00 0
1995-01-10 00:00:00 1
1995-01-11 00:00:00 2
1995-01-12 00:00:00 3


# 시작일과 끝일 반복하면서 평일만 추출

In [17]:
# start_date = datetime(1995,1,3)
start_date = datetime(2025,4,1)
end_date = datetime(2025,4,8)
# print(start_date, end_date)

date_list = []
current_date = start_date

while current_date <= end_date:
    if current_date.weekday() < 5:
        date_list.append(current_date)
    current_date += timedelta(days=1)
        
print(f"총 날짜 수: {len(date_list)}")
print(date_list[:10], end='\n')

총 날짜 수: 6
[datetime.datetime(2025, 4, 1, 0, 0), datetime.datetime(2025, 4, 2, 0, 0), datetime.datetime(2025, 4, 3, 0, 0), datetime.datetime(2025, 4, 4, 0, 0), datetime.datetime(2025, 4, 7, 0, 0), datetime.datetime(2025, 4, 8, 0, 0)]


In [1]:
def new_col(df):
    new_cols = []
    for col in df.columns:
        if col[0] == col[1] == col[2]:
            new_cols.append(col[0].replace(" ", "_"))
        else:
            new_cols.append(" ".join(col).strip().replace(" ", "_"))
    return new_cols

# 코드 합치기

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta
import time
import pandas as pd
from io import StringIO
from ex_dbio import to_exdb

options = Options()
options.add_experimental_option("detach", True)
options.add_argument('start_maximized')
options.add_argument('Chrome/134.0.0.0')
options.add_argument('lang=ko_KR')

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )

url = 'https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_01i.do'
driver.get(url)
wait = WebDriverWait(driver, 10)

start_date = datetime(2025,4,1)
end_date = datetime(2025,4,2)

date_list = []
current_date = start_date

while current_date <= end_date:
    if current_date.weekday() < 5:
        date_list.append(current_date)
    current_date += timedelta(days=1)

    
for date in date_list:
    
    # 날짜 입력
    #tmpInqStrDt
    date_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#tmpInqStrDt')))
    date_input.clear()
    date_input.send_keys(date.strftime('%Y%m%d'))
    date_input.send_keys(Keys.ENTER)

    # 조회버튼 클릭
    #HANA_CONTENTS_DIV > div.btnBoxCenter > a
    search_button = driver.find_element(By.CSS_SELECTOR, '#HANA_CONTENTS_DIV > div.btnBoxCenter > a')
    search_button.click()
    time.sleep(2)
    
    df = pd.read_html(StringIO(driver.find_element(By.CSS_SELECTOR, '.tblBasic.leftNone').get_attribute('outerHTML')))[0]
    df['date'] = date
    
    new_columns = new_col(df)
    df.columns = new_columns
    required_columns = ['date', '통화', '현찰_사실_때_환율', '현찰_사실_때_Spread',
                    '현찰_파실_때_환율', '현찰_파실_때_Spread', '송금_보낼_때_보낼_때',
                    '송금_받을_때_받을_때', '외화_수표_파실때', '매매_기준율', '환가_료율',
                    '미화_환산율']
    df = df[required_columns]

    
    to_exdb(df)
    print(f"{len(df)}개의 환율 데이터프레임을 MySQL DB에 테이블로 저장하였습니다.")

58개의 환율 데이터프레임을 MySQL DB에 테이블로 저장하였습니다.
58개의 환율 데이터프레임을 MySQL DB에 테이블로 저장하였습니다.
